## Condition on mapper

In [1]:
import os
import fire
import random
from retry.api import retry_call
from tqdm import tqdm
from datetime import datetime
from functools import wraps
import sys
sys.path.append("models/stylegan2/")
# from stylegan2 import Trainer, NanException
from csgan2_trainer import Trainer
from util import NanException

import torch
import torch.multiprocessing as mp
import torch.distributed as dist

import numpy as np

from setup import proj_dir, image_dir, out_dir, model_dir

%load_ext autoreload
%autoreload 2


In [2]:
data = image_dir+'zoom15/'
results_dir = out_dir
models_dir = model_dir
name = '2211-ctest5'
new = True
load_from = -1
image_size = 64
condition_dim = 7
condition_on_mapper = False
network_capacity = 16
fmap_max = 512
transparent = False
batch_size = 8
gradient_accumulate_every = 6
num_train_steps = 100000
learning_rate = 2e-4
lr_mlp = 0.1
ttur_mult = 1.5
rel_disc_loss = False
num_workers =  None
save_every = 1000
evaluate_every = 1000
generate = False
num_generate = 1
generate_interpolation = False
interpolation_num_steps = 100
save_frames = False
num_image_tiles = 8
trunc_psi = 0.75
mixed_prob = 0.9
fp16 = False
no_pl_reg = False
cl_reg = False
fq_layers = []
fq_dict_size = 256
attn_layers = []
no_const = False
aug_prob = 0.
aug_types = ['translation', 'cutout']
top_k_training = False
generator_top_k_gamma = 0.99
generator_top_k_frac = 0.5
dual_contrast_loss = False
dataset_aug_prob = 0.
multi_gpus = False
calculate_fid_every = None
calculate_fid_num_images = 12800
clear_fid_cache = False
seed = 42
log = False
rank = 1


In [3]:
model_args = dict(
    name = name,
    results_dir = results_dir,
    models_dir = models_dir,
    batch_size = batch_size,
    gradient_accumulate_every = gradient_accumulate_every,
    image_size = image_size,
    condition_dim = condition_dim,
    condition_on_mapper = condition_on_mapper,
    network_capacity = network_capacity,
    fmap_max = fmap_max,
    transparent = transparent,
    lr = learning_rate,
    lr_mlp = lr_mlp,
    ttur_mult = ttur_mult,
    rel_disc_loss = rel_disc_loss,
    num_workers = num_workers,
    save_every = save_every,
    evaluate_every = evaluate_every,
    num_image_tiles = num_image_tiles,
    trunc_psi = trunc_psi,
    fp16 = fp16,
    no_pl_reg = no_pl_reg,
    cl_reg = cl_reg,
    fq_layers = fq_layers,
    fq_dict_size = fq_dict_size,
    attn_layers = attn_layers,
    no_const = no_const,
    aug_prob = aug_prob,
    aug_types = aug_types,
    top_k_training = top_k_training,
    generator_top_k_gamma = generator_top_k_gamma,
    generator_top_k_frac = generator_top_k_frac,
    dual_contrast_loss = dual_contrast_loss,
    dataset_aug_prob = dataset_aug_prob,
    calculate_fid_every = calculate_fid_every,
    calculate_fid_num_images = calculate_fid_num_images,
    clear_fid_cache = clear_fid_cache,
    mixed_prob = mixed_prob,
    log = log,
    rank = rank,
    demo_channels = 7
)

In [4]:
def run_training(rank, world_size, model_args, data, load_from, new, num_train_steps, name, seed):
    is_main = True #rank == 0
    is_ddp = world_size > 1

    if is_ddp:
        set_seed(seed)
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        dist.init_process_group('nccl', rank=rank, world_size=world_size)

        print(f"{rank + 1}/{world_size} process initialized.")

    model_args.update(
        is_ddp = is_ddp,
        rank = rank,
        world_size = world_size
    )

    model = Trainer(**model_args)
    
    if not new:
        model.load(load_from)
    else:
        model.clear()

    model.set_data_src(data)

    progress_bar = tqdm(initial = model.steps, total = num_train_steps, mininterval=10., desc=f'{name}<{data}>', position=0, leave=True)
    while model.steps < num_train_steps:
        retry_call(model.train, tries=3, exceptions=NanException)
        progress_bar.n = model.steps
        progress_bar.refresh()
        if is_main and model.steps % 50 == 0:
            model.print_log()

    model.save(model.checkpoint_num)

    if is_ddp:
        dist.destroy_process_group()

In [ ]:
rank = 0
world_size = 1
run_training(rank, world_size, model_args, data, load_from, new, num_train_steps, name, seed)

Results directory: /home/jtl/Dropbox (MIT)/project_image_demand/results/sGAN2/2211-ctest5
Model directory: /dreambig/qingyi/image_chicago/models/sGAN2/2211-ctest5
28280 images in dataset
3140 images in dataset


2211-ctest5</dreambig/qingyi/image_chicago/data/images/satellite/zoom15/>:   0%|          | 50/100000 [00:49<27:36:12,  1.01it/s]

G: 13177.37 | D: 0.31 | GP: 24.87


2211-ctest5</dreambig/qingyi/image_chicago/data/images/satellite/zoom15/>:   0%|          | 100/100000 [01:31<25:31:40,  1.09it/s]

G: 10969.26 | D: 5.90 | GP: 159.01


...

2211-ctest5</dreambig/qingyi/image_chicago/data/images/satellite/zoom15/>:  89%|████████▉ | 89450/100000 [21:53:21<2:34:54,  1.14it/s]

G: 0.75 | D: 0.80 | GP: 0.08 | PL: 0.02


2211-ctest5</dreambig/qingyi/image_chicago/data/images/satellite/zoom15/>:  90%|████████▉ | 89500/100000 [21:54:03<2:34:09,  1.14it/s]

G: 0.42 | D: 1.02 | GP: 0.30 | PL: 0.02


2211-ctest5</dreambig/qingyi/image_chicago/data/images/satellite/zoom15/>:  90%|████████▉ | 89531/100000 [21:54:31<2:33:42,  1.14it/s]

In [7]:
model = Trainer(**model_args)


Results directory: /home/jtl/Dropbox (MIT)/project_image_demand/results/sGAN2/2211-ctest5
Model directory: /dreambig/qingyi/image_chicago/models/sGAN2/2211-ctest5


In [22]:
model.load(27)

Loading /dreambig/qingyi/image_chicago/models/sGAN2/2211-ctest/model_27.pt


In [10]:
model.set_data_src(data)

3140 images in dataset
3140 images in dataset


In [24]:
model.evaluate(100)